In [8]:
import os
import gc
import pandas as pd
from sklearn.linear_model import LinearRegression

PATH = '/home/kai/data/kaggle/homecredit/'
if not os.path.exists(PATH + 'inter/linear/'): os.mkdir(PATH + 'inter/linear/')

In [9]:
def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)


def _apply_group(args):
    func, group_id, x, y = args
    return [group_id] + func(x, y)

def apply_by_multiprocessing(func, **kwargs):
    workers = kwargs.pop('workers')
    group_list = kwargs.pop('group_list')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_group, [(func, group_id, x, y) for group_id, x, y in group_list])
    pool.close()
    return result

def liner_reg(x, y, **kwargs):
    lg = LinearRegression()
    lg = LinearRegression()
    lg.fit(x, y)
    z = (x - max(x)) / (max(x) - min(x) + 1)
    lg1.fit(z, y)
    return [lg.coef_[0][0], lg1.coef_[0][0]]

def linear(df_, gp_col, x_col_name, y_col_name, x_map=None, n_job=8):
    df = df_.sort_values(x_col_name).reset_index()
    if x_map != None: df[x_col_name] = df[x_col_name].map(x_map)
    group = [(i, x[x_col_name], x[y_col_name]) for i, x in df.groupby(gp_col)]
    r = apply_by_multiprocessing(liner_reg, workers=n_job, group_list=group)
    tmpdf = pd.DataFrame(r, column=[gp_col, x_col_name+'_lg_'+y_col_name,
                                    x_col_name+'_normallg_'+y_col_name])
    return tmpdf.astype({gp_col: df_[gp_col].dtype})

# bureau

In [ ]:
bureau = pd.read_csv(PATH + 'bureau.csv')
bb = pd.read_csv(PATH + 'bureau_balance.csv')

In [ ]:
tmp_df = pd.DataFrame(bureau[['SK_ID_CURR', 'SK_ID_BUREAU']])
tmp_df1 = bb[bb['STATUS']!='C' & bb['STATUS']!='X']
tmp_df2 = bb[bb['STATUS'] != 'X']
x_map = {'C':0, '0':0, '1':1, '2':2, '3':3, '4':4, '5':5}

tmp_df = tmp_df.merge(linear(tmp_df1, 'SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS', x_map), on='SK_ID_BUREAU', how='left')
tmp_df = tmp_df.merge(linear(tmp_df2, 'SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS', x_map), on='SK_ID_BUREAU', how='left')

r = None
agg_list = ['mean', 'median', 'max', 'min']
for x in tmp_df.columns:
    if x not in ['SK_ID_BUREAU', 'SK_ID_CURR']:
        if r == None: r = numerical(tmp_df, 'SK_ID_CURR', x, agg_list)
        else: r = r.merge(numerical(tmp_df, 'SK_ID_CURR', x, agg_list), on='SK_ID_CURR', how='left')

columns = []
for x in r.columns:
    tmp = 'bureau_' + x if x != merged_col else x
    columns.append(tmp)
r.columns = columns
'done'

In [ ]:
r.to_pickle(PATH + 'inter//linearbureau2curr.pkl')

# installments payments

In [ ]:
inst = pd.read_csv(PATH + 'installments_payments.csv')

In [15]:
name1 = minus_name('DAYS_ENTRY_PAYMENT', 'DAYS_INSTALMENT')
inst[name1] = minus(inst, 'DAYS_ENTRY_PAYMENT', 'DAYS_INSTALMENT')
inst[name1][inst[name1] < 0] = 0
name2 = minus_name('AMT_INSTALMENT', 'AMT_PAYMENT')
inst[name2] = minus(inst, 'AMT_INSTALMENT', 'AMT_PAYMENT')
inst[name2][inst[name2] < 0] = 0
name3 = ratio_name(name2, name1)
inst[name3] = ratio(inst, name2, name1)

r = None
for x in [name1, name2, name3]:
    if r == None: r = linear(inst, 'SK_ID_CURR', x)
    else: r = r.merge(linear(inst, 'SK_ID_CURR', x), on='SK_ID_CURR', how='left')

columns = []
for x in r.columns:
    tmp = 'bureau_' + x if x != merged_col else x
    columns.append(tmp)
r.columns = columns
'done'

9309477 4293019


/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


0


/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
r.to_pickle(PATH + 'inter/linear/linearinstal2curr.pkl')

# credit card balance

# pos cash balance